In [11]:
import sys
import os
import statistics
 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier, AdaBoostClassifier
# Need to upgrade scikit-learn: 0.16.1-np110py34_0 --> 0.17-np110py34_1
from sklearn.cross_validation import cross_val_score

from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.metrics import accuracy_score

In [12]:
def trainAndPredict(clf, trainX, trainY, testX, dimensionReduction = True, n_components = 30):    
    n_train = len(trainX)
    n_test = len(testX)
    X = np.concatenate((trainX, testX), axis=0)
    
    if dimensionReduction:        
        X = preprocessing.scale(X)
        X = PCA(n_components=n_components).fit_transform(X)
    
    trainX = X[0:n_train]
    testX = X[n_train: n_train+n_test+1]
    
    
    if len(trainY.shape) > 1:
        proba = np.zeros((len(testX),3))
        for i in range(trainY.shape[1]):
            clf = clf.fit(trainX,trainY[:,i])
            proba[:,i] = clf.predict_proba(testX)[:,1]
        prediction = np.argmax(proba, axis=1) + 1
    else:
        clf = clf.fit(trainX, trainY)
        proba = clf.predict_proba(testX)
        prediction = clf.predict(testX)

    # Write to file
    results = pd.DataFrame(proba)
    results['prediction'] = prediction
        
    return results

# fill in na according to their labels
def fillna(df, label):
    df_sub = df[df['Y'] == label]
    df_sub = df_sub.fillna(df_sub.median())
    return df_sub

In [13]:
# Deal with input data
trainX = pd.read_csv('trainingData.txt','\t', header = None)
trainX.drop(trainX.columns[len(trainX.columns)-1], axis = 1, inplace = True)
trainY = pd.read_csv("trainingTruth.txt", header = None, names = ['Y'])
df = pd.concat([trainX, trainY], axis=1)
index = df.isnull().sum(axis=1) <= 1
df = df[index]

df1 = fillna(df, 1)
df2 = fillna(df, 2)
df3 = fillna(df, 3)

df = pd.concat([df1, df2, df3])

print('Training data length', len(df))

X = df.iloc[:,0:-1].values
Y = df['Y'].values

Y_binary = np.ones((len(Y),3)) * (-1)
for i in range(3):
    index = Y == (i+1)
    Y_binary[index,i] = 1

testX = pd.read_csv('testData.txt','\t', header = None)
testX.drop(testX.columns[len(testX.columns)-1], axis = 1, inplace = True)
testX.fillna(testX.median(), inplace = True) # Handle NA in test data, although not necessary for this assignment.

# Build classifiers
clf1 = LogisticRegression(random_state=1)
clf2 = RandomForestClassifier(random_state=1, n_estimators=20)
clf3 = GaussianNB()

clf4 = DecisionTreeClassifier(max_depth=4)
clf5 = KNeighborsClassifier(n_neighbors=7)
clf6 = SVC(kernel='rbf', probability=True)
clf7 = AdaBoostClassifier(random_state=1)

eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3),
                                     ('dt', clf4), ('kn', clf5), ('svc', clf6),
                                     ('ab', clf7)], voting='soft')

Training data length 13813


In [21]:
# Get results, write to file, and print out training accuracy
#results_training = trainAndPredict(eclf, X, Y_binary, X)
#print('training accuracy',accuracy_score(Y, results_training['prediction']))

# binary predictions
#results_test= trainAndPredict(clf6, X, Y_binary, testX)

#results_test.to_csv('testY.txt', sep='\t', header = False, index = False)

from sklearn.multiclass import OneVsRestClassifier
results = OneVsRestClassifier(clf1).fit(X, Y).predict_proba(testX)
print(np.sum(results, axis = 1))

[ 1.  1.  1. ...,  1.  1.  1.]


In [16]:
clf1.fit(X, Y).predict_proba(testX)

array([[ 0.01185474,  0.02623709,  0.96190817],
       [ 0.09861403,  0.67431667,  0.2270693 ],
       [ 0.00413735,  0.02134037,  0.97452229],
       ..., 
       [ 0.93826255,  0.0559656 ,  0.00577185],
       [ 0.43262956,  0.47015414,  0.0972163 ],
       [ 0.81808891,  0.18081096,  0.00110012]])

In [22]:
mclf = OneVsRestClassifier(clf1)

# Cross validation
from sklearn.cross_validation import ShuffleSplit,KFold

ss = ShuffleSplit(n=len(Y), n_iter=10, test_size=0.2, random_state=1)
#kf = KFold(len(Y), n_folds=10)

In [24]:
scores = []
for train, test in ss:
    results = mclf.fit(X[train], Y[train]).predict(X[test])
    score = accuracy_score(Y[test], results)
    scores.append(score)
    print(score)
                  
print(np.mean(scores),np.std(scores))

0.848715164676
0.849077090119
0.837495475932
0.846543612016
0.853058269996
0.849800941006
0.844372059356
0.851610568223
0.847629388346
0.847629388346
0.847593195802 0.0040961582612


In [25]:
scores = []
for train, test in ss:
    results = clf1.fit(X[train], Y[train]).predict(X[test])
    score = accuracy_score(Y[test], results)
    scores.append(score)
    print(score)
                  
print(np.mean(scores),np.std(scores))

0.848715164676
0.849077090119
0.837495475932
0.846543612016
0.853058269996
0.849800941006
0.844372059356
0.851610568223
0.847629388346
0.847629388346
0.847593195802 0.0040961582612


In [26]:
# Cross validation
from sklearn.cross_validation import ShuffleSplit,KFold

#ss = ShuffleSplit(n=len(Y), n_iter=10, test_size=0.2, random_state=1)
#kf = KFold(len(Y), n_folds=10)
scores = []
for train, test in ss:
    results = trainAndPredict(clf1, X[train], Y_binary[train], X[test])
    score = accuracy_score(Y[test], results['prediction'])
    scores.append(score)
    print(score)
                  
print(np.mean(scores),np.std(scores))

0.844372059356
0.85884907709
0.835685848715
0.844733984799
0.846543612016
0.846543612016
0.846181686573
0.852696344553
0.845095910242
0.847629388346
0.846833152371 0.00563674913758


In [74]:
import numpy as np
from sklearn import metrics
from __future__ import division
y = np.array([1, 1, 2, 2])
pred = np.array([0.8, 0.8, 0.9, 1])
fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=2)
score = metrics.auc(fpr, tpr)
print(fpr, tpr, thresholds, score)

[ 0.  0.  1.] [ 0.5  1.   1. ] [ 1.   0.9  0.8] 1.0
